### TO DO:

- correct the path for the pipeline
- correct the pathes


In [ ]:
import os
import time
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage.transform import resize
from skimage import data, exposure
from PIL import Image
from sklearn.svm import SVC #SVM classifier
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import local_binary_pattern
from skimage.color import rgb2gray

In [ ]:
def normalize_and_flatten(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features.flatten()


In [ ]:
resize_factor = (188, 250)

train_img_path = '../data/train_set/train_images/'
test_img_path = '../data/train_set/test_images/'
train_features_path = '../data/train_set/train_points/'
test_features_path = '../data/train_set/test_points/'
labels = pd.read_csv('../data/train_set/label.csv')

train_img = sorted(os.listdir(train_img_path))
test_img = sorted(os.listdir(test_img_path))

## TRUE Labels

In [ ]:
### Train emotion indexes
train_indexes = []
for img in train_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    train_indexes.append(img - 1)
    
emotion_idx = labels[['emotion_idx']].loc[train_indexes].values[:, 0]

### Test emotion indexes
test_indexes = []
for img in test_img:
    if img == ".DS_Store":
        continue
    img = int(img[:4])
    test_indexes.append(img - 1)
    
test_emotion_idx = labels[['emotion_idx']].loc[test_indexes].values[:, 0]   

## FEATURES
#### HOGS, train_features, concat_train_features

In [ ]:
### extract HOG for train data set

HOGs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time))  

### extract HOG for test data set

HOGs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    fetures_hog = hog(img, orientations=9, pixels_per_cell=(8, 8),
                    cells_per_block=(3, 3), visualize=False, multichannel=True)
    HOGs_test.append(fetures_hog)
print("--- %s seconds ---" % (time.time() - start_time)) 


In [ ]:
### Point features for train dataset (NORMALIZED VERSION)
train_features = sorted(os.listdir(train_features_path))
train_features_list = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list.append(normalize_and_flatten(features))
    
### Point features for test dataset (NORMALIZED VERSION)
test_features = sorted(os.listdir(test_features_path))
test_features_list = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list.append(normalize_and_flatten(features))    

In [ ]:
test_features_list[1].shape

In [ ]:
### Cancate of HOG and .mat features train
start_time = time.time()
pca = PCA(n_components=100)
reduced_hogs = pca.fit_transform(np.array(HOGs)) 
train_np = np.array(train_features_list)
concat_train_features = np.concatenate((reduced_hogs, train_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

### test
start_time = time.time()
reduced_hogs_test = pca.transform(np.array(HOGs_test))
test_np = np.array(test_features_list)
concat_test_features = np.concatenate((reduced_hogs_test, test_np), axis=1)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
### extract LBP for train data set

METHOD = 'uniform'
radius = 3
n_points = 8 * radius

LBPs = []
start_time = time.time()
for img in train_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join(train_img_path, img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    img = rgb2gray(img)
#     print(img.shape)    
    fetures_lbp = local_binary_pattern(img, n_points, radius, METHOD)
    LBPs.append(fetures_lbp.flatten())

print("--- %s seconds ---" % (time.time() - start_time))  

### extract LBP for test data set

LBPs_test = []
start_time = time.time()
for img in test_img:
    if img == '.DS_Store':
        continue
    img = np.asarray(Image.open(os.path.join('../data/train_set/test_images/', img)))
    img = resize(img, resize_factor, anti_aliasing=False)
    img = rgb2gray(img)
    fetures_lbp = local_binary_pattern(img, n_points, radius, METHOD)
    LBPs_test.append(fetures_lbp.flatten())
print("--- %s seconds ---" % (time.time() - start_time)) 


## CLASSIFIERS

#### Linear SVM on HOG

In [ ]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(HOGs), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(HOGs_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### Linear SVM on LBP

In [ ]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.001)
linear_SVM.fit(np.array(LBPs), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(LBPs), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(LBPs_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### Linear SVM on PCA on LBP

In [ ]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
linear_SVM = SVC(kernel="linear", C=0.0001)
linear_SVM.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### Linear SVM on .mat features

In [ ]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=100)
linear_SVM.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### L SVM on concat features

In [ ]:
start_time = time.time()
linear_SVM = SVC(kernel="linear", C=0.5)
linear_SVM.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = linear_SVM.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### L SVM on PCA HOG

In [ ]:
start_time = time.time()
# pca = PCA(n_components=200)
# reduced_features = pca.fit_transform(np.array(HOGs))
linear_SVM = SVC(kernel="linear", C=1)
linear_SVM.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = linear_SVM.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = linear_SVM.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### SVM on PCA HOG

In [ ]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = SVC(gamma='auto', C=10)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = SVC(gamma='auto', C=0.01)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### SVM on .mat Features

In [ ]:
start_time = time.time()
model = SVC(gamma='auto', C=1000)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### SVM on concat features

In [ ]:
start_time = time.time()
model = SVC(gamma='auto', C=10)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### RandomForestClassifier on PCA HOG

In [ ]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = RandomForestClassifier(max_depth=5, n_estimators=50)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### RFC on .mat features

In [ ]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### RFC on cancate features

In [ ]:
start_time = time.time()
model = RandomForestClassifier(max_depth=5, n_estimators=100)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### MLPClassifier on pca HOG

In [ ]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### MLP on .mat

In [ ]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 150, 60, 40))
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### MLP on concat features

In [ ]:
start_time = time.time()
model = MLPClassifier(hidden_layer_sizes=(300, 600, 300, 60, 40))
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### KNN on .mat features

In [ ]:
start_time = time.time()
model = KNeighborsClassifier(n_neighbors=5)
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### KNN on pca HOG

In [ ]:
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(HOGs))
model = KNeighborsClassifier(n_neighbors=10)
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(HOGs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

#### KNN on concate features

In [ ]:
start_time = time.time()
model = KNeighborsClassifier(n_neighbors=3)
model.fit(concat_train_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(concat_train_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(concat_test_features, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

start_time = time.time()
model = AdaBoostClassifier(n_estimators=150)
scores = cross_val_score(model, np.array(train_features_list), emotion_idx, cv=5)
print("Train Duration : %s seconds" % (time.time() - start_time)) 
print(scores)

# model.fit(np.array(train_features_list), emotion_idx)
# print("Train Duration : %s seconds" % (time.time() - start_time)) 

# start_time = time.time()
# accuracy = model.score(np.array(train_features_list), emotion_idx)
# print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
# print("Train Accuracy: {}".format(accuracy))

# start_time = time.time()
# accuracy = model.score(np.array(test_features_list), test_emotion_idx)
# print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
# print("Test Accuracy: {}".format(accurac

In [ ]:
lbp.shape

In [ ]:
from skimage.transform import rotate
from skimage.feature import local_binary_pattern
from skimage import data
from skimage.color import label2rgb

# settings for LBP
radius = 3
n_points = 8 * radius


def overlay_labels(image, lbp, labels):
    mask = np.logical_or.reduce([lbp == each for each in labels])
    return label2rgb(mask, image=image, bg_label=0, alpha=0.5)


def highlight_bars(bars, indexes):
    for i in indexes:
        bars[i].set_facecolor('r')


image = data.brick()

In [ ]:
image.shape

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

start_time = time.time()
model = LDA()
model.fit(np.array(train_features_list), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(train_features_list), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(test_features_list), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
start_time = time.time()
pca = PCA(n_components=200)
reduced_features = pca.fit_transform(np.array(LBPs))
model = LDA()
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(LBPs_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
### calculating the distance
def normalize(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features

train_features = sorted(os.listdir(train_features_path))
train_features_list2 = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list2.append(normalize(features))
    
test_features = sorted(os.listdir(test_features_path))
test_features_list2 = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list2.append(normalize(features)) 

In [ ]:
train_features_list2[0].shape

In [ ]:
import math
def euclideanـdistance(p1, p2):
    tmp = ((p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1]))
    return math.sqrt(tmp)

dist = []
for i, p1 in enumerate(train_features_list2[0]):
    for j, p2 in enumerate(train_features_list2[0]):
        if j > i:
            dist.append(euclideanـdistance(p1, p2))

In [ ]:
euclideanـistance_train = []
euclideanـdistance_test = []

start_time = time.time()
for im in train_features_list2:
    dist = []
    for i, p1 in enumerate(im):
        for j, p2 in enumerate(im):
            if j > i:
                dist.append(euclideanـdistance(p1, p2))
    euclideanـdistance_train.append(dist)    
print("Duration: %s seconds" % (time.time() - start_time)) 
   
start_time = time.time()
for im in test_features_list2:
    dist = []
    for i, p1 in enumerate(im):
        for j, p2 in enumerate(im):
            if j > i:
                dist.append(euclideanـdistance(p1, p2))
    euclideanـdistance_test.append(dist)    
print("Duration: %s seconds" % (time.time() - start_time)) 
      

In [ ]:
len(euclideanـdistance_train[0])

In [ ]:
start_time = time.time()
model = LDA()
model.fit(np.array(euclideanـdistance_train), emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(np.array(euclideanـdistance_train), emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
accuracy = model.score(np.array(euclideanـdistance_test), test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
### euclideanـistance and pca and lda 
start_time = time.time()
pca = PCA(n_components=100)
reduced_features = pca.fit_transform(np.array(euclideanـdistance_train))
model = LDA()
model.fit(reduced_features, emotion_idx)
print("Train Duration : %s seconds" % (time.time() - start_time)) 

start_time = time.time()
accuracy = model.score(reduced_features, emotion_idx)
print("Train Accuracy Durtion: %s seconds" % (time.time() - start_time)) 
print("Train Accuracy: {}".format(accuracy))

start_time = time.time()
reduced_test = pca.transform(np.array(euclideanـdistance_test))
accuracy = model.score(reduced_test, test_emotion_idx)
print("Test Accuracy Duration: %s seconds" % (time.time() - start_time)) 
print("Test Accuracy: {}".format(accuracy))

In [ ]:
import pandas as pd 
pd.DataFrame(reduced_features).to_csv("PCA_Euclidean_Features_Train.csv")
pd.DataFrame(reduced_test).to_csv("PCA_Euclidean_Features_Test.csv")


In [ ]:
train_pd = pd.DataFrame(reduced_features)
test_pd = pd.DataFrame(reduced_test)

train_pd['emotion_idx'] = emotion_idx

test_pd['emotion_idx'] = test_emotion_idx

In [ ]:
train_pd.to_csv("PCA_Euclidean_Features_Train.csv")
test_pd.to_csv("PCA_Euclidean_Features_Test.csv")

In [ ]:
#### distance is going to be the pairwise absolute value as a pca, and the lda 

In [ ]:
### calculating the distance
def normalize(features):
    features = features - np.min(features, axis=0, keepdims=True)
    features = features / np.max(features, axis=0, keepdims=True)
    return features

train_features = sorted(os.listdir(train_features_path))
train_features_list2 = []
for f in train_features:
    tmp = sio.loadmat(os.path.join(train_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    train_features_list2.append(normalize(features))
    
test_features = sorted(os.listdir(test_features_path))
test_features_list2 = []
for f in test_features:
    tmp = sio.loadmat(os.path.join(test_features_path, f))
    if 'faceCoordinatesUnwarped' in tmp:
        features = tmp['faceCoordinatesUnwarped']
    else:
        features = tmp['faceCoordinates2']
    test_features_list2.append(normalize(features)) 

In [ ]:
import math
def pairwise_distance(p1, p2):
    return abs(p1[0] - p2[0]), abs(p1[1] - p2[1])

dist = []
for i, p1 in enumerate(train_features_list2[0]):
    for j, p2 in enumerate(train_features_list2[0]):
        if j > i:
            dist.append(pairwise_distance(p1, p2))      